## Homework 2 -  Classification
***
**Name**: $<$insert name here$>$ 
***

Remember that you are encouraged to discuss the problems with your instructors and classmates, but **you must write all code and solutions on your own**.

The rules to be followed for the assignment are:

- Do **NOT** load additional packages beyond what we've shared in the cells below.
- Some problems with code may be autograded.  If we provide a function or class API **do not** change it.
- Do not change the location of the data or data directory.  Use only relative paths to access the data.

In [15]:
import argparse
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from collections import defaultdict

### [10 points] Problem 1 - Building a Decision Tree
***

A sample dataset has been provided to you in the './data/dataset.csv' path. Here are the attributes for the dataset. Use this dataset to test your functions.

- Age - ["<=30", "31-40", ">40"]
- Income - ["low", "medium", "high"]
- Student - ["no", "yes"]
- Credit Rating - ["fair", "excellent"]
- Loan - ["no", "yes"]

Note:
- A sample dataset to test your code has been provided in the location "data/dataset.csv". Please maintain this as it would be necessary while grading.
- Do not change the variable names of the returned values.
- After calculating each of those values, assign them to the corresponding value that is being returned.
- The "Loan" attribute should be used as the target variable while making calculations for your decision tree.

In [24]:
## THIS IS CORRECT IF CHANGED GRADE WILL CHANGE FROM 15/20 TO 10/20

import math
import pandas as pd

def information_gain_target(dataset_file): 
    
#        Input: dataset_file - A string variable which references the path to the dataset file.
#        Output: ig_loan - A floating point variable which holds the information entropy associated with the target variable.
#        
#        NOTE: 
#        1. Return the information gain associated with the target variable in the dataset.
#        2. The Loan attribute is the target variable
#        3. The pandas dataframe has the following attributes: Age, Income, Student, Credit Rating, Loan
#        4. Perform your calculations for information gain and assign it to the variable ig_loan


# TARGET VARIABLE IS THE LOAN ATTRIBUTE
# Information gain is Gain(A) = Info(D) - Info subsetA (D)

    df = pd.read_csv(dataset_file)
    ig_loan = 0
    
    # your code here
    target_attribute = 'Loan'
    
    # Calculate the total entropy of the target variable
    total_records = len(df)
    loan_counts = df[target_attribute].value_counts()
    loan_probabilities = loan_counts / total_records
    loan_entropy = sum([-p * math.log2(p) for p in loan_probabilities])

    # Calculate the information gain for each attribute
    attribute_columns = df.columns[:-1]  # Exclude the target variable column
    attribute_entropies = []

    for attribute in attribute_columns:
        attribute_values = df[attribute].unique()
        attribute_entropy = 0

        for value in attribute_values:
            subset = df[df[attribute] == value]
            subset_records = len(subset)
            subset_loan_counts = subset[target_attribute].value_counts()
            subset_loan_probabilities = subset_loan_counts / subset_records
            subset_loan_entropy = sum([-p * math.log2(p) for p in subset_loan_probabilities])

            attribute_entropy += (subset_records / total_records) * subset_loan_entropy

        attribute_entropies.append(attribute_entropy)

    # Calculate the information gain for the target variable
    ig_loan = loan_entropy - sum(attribute_entropies)
    
    return ig_loan


information_gain_target('data/dataset.csv')

-2.417706237779429

In [25]:

# This cell has hidden test cases that will run after you submit your assignment. 


In [26]:
attribute_values = {
    "Age": ["<=30", "31-40", ">40"],
    "Income": ["low", "medium", "high"],
    "Student": ["yes", "no"],
    "Credit Rating": ["fair", "excellent"]
}

attributes = ["Age", "Income", "Student", "Credit Rating"]

In [27]:
import math

def information_gain(p_count_yes, p_count_no):
    # Input: p_count_yes - Number of instances with target value 'yes'
    #        p_count_no - Number of instances with target value 'no'
    # Output: ig - Information gain value
    
    total_count = p_count_yes + p_count_no
    p_yes = p_count_yes / total_count
    p_no = p_count_no / total_count

    if p_yes == 0 or p_no == 0:
        return 0

    ig = -(p_yes * math.log2(p_yes)) - (p_no * math.log2(p_no))

    return ig


information_gain(30, 70)

0.8812908992306927

In [20]:
import operator

def information_gain_attributes(dataset_file, ig_loan, attributes, attribute_values):
    results = {
        "ig_attributes": {
            "Age": 0,
            "Income": 0,
            "Student": 0,
            "Credit Rating": 0
        },
        "best_attribute": ""
    }

    df = pd.read_csv(dataset_file)
    d_range = len(df)

    for attribute in attributes:
        ig_attribute = 0
        value_counts = dict()
        vcount = df[attribute].value_counts()
        for att_value in attribute_values[attribute]:
            # YOUR CODE HERE
            count_yes = df.loc[(df[attribute] == att_value) & (df["Loan"] == "yes")].shape[0]
            count_no = df.loc[(df[attribute] == att_value) & (df["Loan"] == "no")].shape[0]
            total_count = count_yes + count_no
            p_att_value = total_count / d_range
            p_count_yes = count_yes / total_count if total_count != 0 else 0
            p_count_no = count_no / total_count if total_count != 0 else 0
            value_counts[att_value] = total_count
            ig_attribute += p_att_value * information_gain(p_count_yes, p_count_no)

        results["ig_attributes"][attribute] = ig_loan - ig_attribute

    results["best_attribute"] = max(results["ig_attributes"].items(), key=operator.itemgetter(1))[0]
    return results



information_gain_attributes("data/dataset.csv", -2.417, attributes, attribute_values)


{'ig_attributes': {'Age': -3.1548960810227786,
  'Income': -3.3844700395364007,
  'Student': -3.2011591278514215,
  'Credit Rating': -3.32504974601998},
 'best_attribute': 'Age'}

In [21]:

# This cell has hidden test cases that will run after you submit your assignment. 


### [10 points] Problem 2 - Building a Naive Bayes Classifier
***

A sample dataset has been provided to you in the './data/dataset.csv' path. Here are the attributes for the dataset. Use this dataset to test your functions.

- Age - ["<=30", "31-40", ">40"]
- Income - ["low", "medium", "high"]
- Student - ["no", "yes"]
- Credit Rating - ["fair", "excellent"]
- Loan - ["no", "yes"]

Note:
- A sample dataset to test your code has been provided in the location "data/dataset.csv". Please maintain this as it would be necessary while grading.
- Do not change the variable names of the returned values.
- After calculating each of those values, assign them to the corresponding value that is being returned.
- The "Loan" attribute should be used as the target variable while making calculations for your naive bayes classifier.

In [22]:
from collections import defaultdict

def naive_bayes(dataset_file, attributes, attribute_values):

#   Input:
#       1. dataset_file - A string variable which references the path to the dataset file.
#       2. attributes - A python list which has all the attributes of the dataset
#       3. attribute_values - A python dictionary representing the values each attribute can hold.
#        
#   Output: A probabilities dictionary which contains the values of when the input attribute is yes or no
#       depending on the corresponding Loan attribute.
#                
#   Hint: Starter code has been provided to you to calculate the probabilities.

    probabilities = {
        "Age": { "<=30": {"yes": 0, "no": 0}, "31-40": {"yes": 0, "no": 0}, ">40": {"yes": 0, "no": 0} },
        "Income": { "low": {"yes": 0, "no": 0}, "medium": {"yes": 0, "no": 0}, "high": {"yes": 0, "no": 0}},
        "Student": { "yes": {"yes": 0, "no": 0}, "no": {"yes": 0, "no": 0} },
        "Credit Rating": { "fair": {"yes": 0, "no": 0}, "excellent": {"yes": 0, "no": 0} },
        "Loan": {"yes": 0, "no": 0}
    }
    
    df = pd.read_csv(dataset_file)
    d_range = len(df)
    
    
    vcount = df["Loan"].value_counts()
    vcount_loan_yes = vcount["yes"]
    vcount_loan_no = vcount["no"]
    
    probabilities["Loan"]["yes"] = vcount_loan_yes/d_range
    probabilities["Loan"]["no"] = vcount_loan_no/d_range
    
    for attribute in attributes:
        value_counts = dict()
        vcount = df[attribute].value_counts()
        
        for att_value in attribute_values[attribute]:    
        # your code here

            
            
            
##### THIS CODE IS CORRECT - WHEN IT IS CHANGED SCORE GOES FROM 10/20 TO 0/20    
        
        
            filtered_df_yes = df[df["Loan"] == "yes"]
            num_yes = len(filtered_df_yes[filtered_df_yes[attribute] == att_value])
            filtered_range_yes = len(filtered_df_yes)
            
            filtered_df_no = df[df["Loan"] == "no"]
            num_no = len(filtered_df_no[filtered_df_no[attribute] == att_value])
            filtered_range_no = len(filtered_df_no)
            
            
            probabilities[attribute][att_value]["yes"] = num_yes/filtered_range_yes
            probabilities[attribute][att_value]["no"] = num_no/filtered_range_no
    
    return probabilities

naive_bayes("data/dataset.csv", attributes, attribute_values)

{'Age': {'<=30': {'yes': 0.2857142857142857, 'no': 0.6},
  '31-40': {'yes': 0.42857142857142855, 'no': 0.0},
  '>40': {'yes': 0.2857142857142857, 'no': 0.4}},
 'Income': {'low': {'yes': 0.2857142857142857, 'no': 0.2},
  'medium': {'yes': 0.42857142857142855, 'no': 0.4},
  'high': {'yes': 0.2857142857142857, 'no': 0.4}},
 'Student': {'yes': {'yes': 0.7142857142857143, 'no': 0.2},
  'no': {'yes': 0.2857142857142857, 'no': 0.8}},
 'Credit Rating': {'fair': {'yes': 0.7142857142857143, 'no': 0.4},
  'excellent': {'yes': 0.2857142857142857, 'no': 0.6}},
 'Loan': {'yes': 0.5833333333333334, 'no': 0.4166666666666667}}

In [11]:

# This cell has hidden test cases that will run after you submit your assignment. 
